<a href="https://colab.research.google.com/github/Deni1019/Bootcamp-Caracteristicas-del-mercado-energ-tico/blob/main/limpieza_de_datos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [43]:
#importar librerias
import pandas as pd
import os
import re

In [44]:
#acceder a los datos a traves de Google Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [45]:
#Definir las rutas de acceso a los archivos
ruta_demanda = '/content/drive/MyDrive/Bootcamp 2025/datos_energeticos/demanda_energetica'
ruta_produccion = '/content/drive/MyDrive/Bootcamp 2025/datos_energeticos/produccion_energetica'
ruta_diccionario = '/content/drive/MyDrive/Bootcamp 2025/datos_energeticos/produccion_energetica/diccionario_plantas_departamento.xlsx'

In [46]:
diccionario = pd.read_excel(ruta_diccionario)
diccionario.columns = diccionario.iloc[6]
diccionario = diccionario.drop(index = range(0,10))
diccionario = diccionario.reset_index(drop=True)
print("DataFrame original:")
print(diccionario.head())
print('_______________________________')
print('Serie de planta vs dpto.')
clave = diccionario[['Tipo/Fuente de energía','Departamento']]
print(clave.head())
print('_______________________________')
print('Serie filtrada')
mascara = clave['Departamento'].str.strip() != '-'
clave_filtrada = clave[mascara]
print(clave_filtrada.head())
print(f'Tamaño de la serie filtrada: {clave_filtrada.shape}')

DataFrame original:
6 Tipo/Fuente de energía Capacidad efectiva neta [MW]  \
0                 AMALFI                         0,81   
1                AMERICA                         0,41   
2                 ASNAZU                         0,45   
3                 BAYONA                          0,6   
4                  BELLO                         0,35   

6                              Operador Fecha de puesta en operación FPO  \
0               HZ ENERGY S.A.S. E.S.P.                       05/08/2007   
1  EMPRESAS PUBLICAS DE MEDELLIN E.S.P.                       01/01/1997   
2                     VATIA S.A. E.S.P.                       01/01/1934   
3      EMPRESAS PUBLICAS DE CALARCA ESP                       01/01/1943   
4  EMPRESAS PUBLICAS DE MEDELLIN E.S.P.                       01/01/1997   

6 Municipio Departamento            Subárea  
0    AMALFI    ANTIOQUIA  SubArea Antioquia  
1  MEDELLÍN    ANTIOQUIA  SubArea Antioquia  
2    SUÁREZ        CAUCA      SubArea Vall

In [47]:
clave_filtrada.rename(columns={'Tipo/Fuente de energía': 'Planta'}, inplace=True)
diccionario_plantas = clave_filtrada
diccionario_plantas.head()

/tmp/ipython-input-3767197889.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clave_filtrada.rename(columns={'Tipo/Fuente de energía': 'Planta'}, inplace=True)


6,Planta,Departamento
0,AMALFI,ANTIOQUIA
1,AMERICA,ANTIOQUIA
2,ASNAZU,CAUCA
3,BAYONA,QUINDÍO
4,BELLO,ANTIOQUIA


In [48]:
#Conseguir la información de produccion por departamento
lista_df_produccion = []
for archivo in os.listdir(ruta_produccion):
    if archivo.startswith('produccion') and archivo.endswith('.xlsx'):

        print(f"Archivo válido encontrado, procesando: {archivo}")
        ruta_completa = os.path.join(ruta_produccion, archivo)

        df_temporal = pd.read_excel(ruta_completa, header=2)

        año = re.search(r'(\d{4})', archivo).group(1)
        df_temporal['Año'] = año

        lista_df_produccion.append(df_temporal)

df_produccion_total = pd.concat(lista_df_produccion, ignore_index=True)

print("\n¡Proceso completado!")
print(f"Se procesaron {len(lista_df_produccion)} archivos de producción.")
print(df_produccion_total.head())



Archivo válido encontrado, procesando: produccion2016.xlsx
Archivo válido encontrado, procesando: produccion2017.xlsx
Archivo válido encontrado, procesando: produccion2018.xlsx
Archivo válido encontrado, procesando: produccion2019.xlsx
Archivo válido encontrado, procesando: produccion2020.xlsx
Archivo válido encontrado, procesando: produccion2022.xlsx
Archivo válido encontrado, procesando: produccion2023.xlsx
Archivo válido encontrado, procesando: produccion2024.xlsx
Archivo válido encontrado, procesando: produccion2015.xlsx
Archivo válido encontrado, procesando: produccion2021.xlsx

¡Proceso completado!
Se procesaron 10 archivos de producción.
       Planta Tipo Despacho  Generación [GWh]  Part.[%]   Año
0      GUAVIO            DC       5948.994911  0.090215  2016
1  SAN CARLOS            DC       5009.320864  0.075965  2016
2      CHIVOR            DC       4305.217742  0.065288  2016
3      TEBSAB            DC       4299.020467  0.065194  2016
4       PAGUA            DC       396

In [49]:
#El dataframe filtrado será
produccion_dpto = df_produccion_total[['Año','Planta','Generación [GWh]']]
print(produccion_dpto.head())
print(produccion_dpto.shape)

    Año      Planta  Generación [GWh]
0  2016      GUAVIO       5948.994911
1  2016  SAN CARLOS       5009.320864
2  2016      CHIVOR       4305.217742
3  2016      TEBSAB       4299.020467
4  2016       PAGUA       3962.608120
(3466, 3)


In [51]:
print(diccionario_plantas.head(10))
print(len(diccionario_plantas))

6                       Planta     Departamento
0                       AMALFI        ANTIOQUIA
1                      AMERICA        ANTIOQUIA
2                       ASNAZU            CAUCA
3                       BAYONA          QUINDÍO
4                        BELLO        ANTIOQUIA
5          CAMPESTRE (CALARCA)          QUINDÍO
6              CAMPESTRE (EPM)        ANTIOQUIA
7                    EL COCUYO  VALLE DEL CAUCA
8                     GUACAICA           CALDAS
9                   INTERMEDIA           CALDAS
314


In [52]:
#Asignar estas plantas manualmente hasta masomenos llegar a un estimativo razonable

plantas_nuevas = ['PAGUA', 'GUATRON', 'ALBAN', 'FLORES 4B', 'TERMOSIERRAB', 'FLORES 1']
departamentos_nuevas = ['CUNDINAMARCA', 'ANTIOQUIA', 'VALLE DEL CAUCA', 'ATLÁNTICO', 'ANTIOQUIA', 'ATLÁNTICO']
plantas_nuevas_dict = {'Planta': plantas_nuevas, 'Departamento': departamentos_nuevas}
df_diccionario = pd.DataFrame(plantas_nuevas_dict)

In [68]:
def crear_llave_normalizada(nombre_planta):
    if not isinstance(nombre_planta, str):
        return ""
    texto = nombre_planta.lower()
    texto = texto.replace('á', 'a').replace('é', 'e').replace('í', 'i').replace('ó', 'o').replace('ú', 'u')
    texto = re.sub(r'[^a-z0-9\s]', '', texto)
    palabras_a_quitar = ['planta', 'central', 'sa', 'esp', 'cc', 'sas', 'ltda', '1', '2', 'autog', 'autg', 'agpe']
    for palabra in palabras_a_quitar:
        texto = re.sub(r'\b' + palabra + r'\b', '', texto)

    texto = re.sub(r'\s+', '', texto)
    return texto.strip()

produccion_dpto['llave_planta'] = produccion_dpto['Planta'].apply(crear_llave_normalizada)
diccionario_plantas['llave_planta'] = diccionario_plantas['Planta'].apply(crear_llave_normalizada)

lista_plantas_diccionario = diccionario_plantas['llave_planta'].unique()
plantas_no_encontradas = produccion_dpto[~produccion_dpto['llave_planta'].isin(lista_plantas_diccionario)]

ranking_faltantes = plantas_no_encontradas.groupby('Planta')['Generación [GWh]'].sum().reset_index()
ranking_faltantes_ordenado = ranking_faltantes.sort_values(by='Generación [GWh]', ascending=False)

print(f'Plantas no encontradas {plantas_no_encontradas.shape}')
print("--- Ranking de Plantas Faltantes por Impacto (Top 20) ---")
print(ranking_faltantes_ordenado.head(20))

Plantas no encontradas (1273, 4)
--- Ranking de Plantas Faltantes por Impacto (Top 20) ---
                          Planta  Generación [GWh]
548                   EL MORRO 1        937.470253
542                     CIMARRON        851.053237
549                   EL MORRO 2        825.937034
564             JEPIRACHI 1 - 15        373.227925
575      PARQUE EOLICO GUAJIRA I        213.346203
602             TRINA-VATIA BSLI        189.875826
603            TRINA-VATIA BSLII        177.441325
604           TRINA-VATIA BSLIII        174.781883
569                    LA SIERPE        144.854612
576         PARQUE EÓLICO WESP01        139.530813
543       COGENERADOR COLTEJER 1        118.982442
594                        SINCE        107.105219
528         CELSIA SOLAR BOLIVAR         91.974119
531          CELSIA SOLAR DULIMA         87.179215
553         GRANJA SOLAR FLANDES         83.813653
570                     LA TOLUA         78.761940
508        AUTOG ECOPETROL ORITO         7

/tmp/ipython-input-4117111255.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  produccion_dpto['llave_planta'] = produccion_dpto['Planta'].apply(crear_llave_normalizada)


In [69]:
#Cálculo del Porcentaje Asignado
generacion_total = produccion_dpto['Generación [GWh]'].sum()
generacion_no_asignada = plantas_no_encontradas['Generación [GWh]'].sum()
porcentaje_no_asignado = (generacion_no_asignada / generacion_total) * 100

print(f"Hay {len(plantas_no_encontradas)} plantas que no fueron asignadas a un dpto")
print(f"Porcentaje de energía NO ASIGNADA: {porcentaje_no_asignado:.2f}%")
print(f"Porcentaje de energía que SÍ FUE ASIGNADA a un departamento: {100 - porcentaje_no_asignado:.2f}%")

Hay 1273 plantas que no fueron asignadas a un dpto
Porcentaje de energía NO ASIGNADA: 0.87%
Porcentaje de energía que SÍ FUE ASIGNADA a un departamento: 99.13%


In [55]:
ranking_faltantes_ordenado[:21]

,Planta,Generación [GWh]
579,PAGUA,32382.893810
562,GUATRON,26247.218855
492,ALBAN,16022.102605
552,FLORES 4B,7947.908399
605,TERMOSIERRAB,3284.174978
551,FLORES 1,2469.962864
549,EL MORRO 1,937.470253
543,CIMARRON,851.053237
550,EL MORRO 2,825.937034
568,JEPIRACHI 1 - 15,373.227925


In [56]:
#Concatenar los resultados al diccionario por dpto
diccionario_plantas = pd.concat([diccionario_plantas, df_diccionario], ignore_index=True)

In [57]:
# Unimos la tabla de producción con el diccionario usando la llave normalizada.
produccion_con_dpto = pd.merge(
    produccion_dpto,
    diccionario_plantas,
    on='llave_planta',
    how='left'
)

produccion_con_dpto['Departamento'].fillna('No Asignado / Menores', inplace=True)

produccion_agrupada = produccion_con_dpto.groupby(['Año', 'Departamento'])['Generación [GWh]'].sum().reset_index()

produccion_final = produccion_agrupada.rename(columns={
    'Departamento': 'Departamento',
    'Generación [GWh]': 'Produccion_GWh'
})

print("DataFrame Final: Producción por Año y Departamento")
print(produccion_final.head(10))

DataFrame Final: Producción por Año y Departamento
    Año  Departamento  Produccion_GWh
0  2015     ANTIOQUIA    16495.333354
1  2015     ATLÁNTICO     5389.818127
2  2015  BOGOTÁ, D.C.       20.204581
3  2015       BOLÍVAR     1205.155267
4  2015        BOYACÁ     7394.486413
5  2015        CALDAS     1568.395504
6  2015      CASANARE      818.884378
7  2015         CAUCA     1000.681798
8  2015  CUNDINAMARCA     8318.745956
9  2015       CÓRDOBA     1440.367882


/tmp/ipython-input-1777290254.py:9: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  produccion_con_dpto['Departamento'].fillna('No Asignado / Menores', inplace=True)


The error `IsADirectoryError: [Errno 21] Is a directory: '/content/drive/MyDrive/archivos energeticos/demanda_energetica'` occurs because you are trying to use `pd.read_excel` on a directory path (`ruta_demanda`) instead of a file path. Inside the loop, you should be reading each individual Excel file found in the directory. Also, you are currently appending the temporary dataframes to `lista_df_produccion` instead of `lista_df_demanda`.

I will provide a corrected version of the code for cell `huk8Hqx-0BaO` to fix these issues.

In [58]:
#Leer y agrupar la demanda energetica
lista_df_demanda = []
for archivo in os.listdir(ruta_demanda):
    if archivo.startswith('demanda') and archivo.endswith('.csv'):

        print(f"Archivo válido encontrado, procesando: {archivo}")
        ruta_completa = os.path.join(ruta_demanda, archivo)

        df_temporal = pd.read_csv(ruta_completa)

        año = re.search(r'(\d{4})', archivo).group(1)
        df_temporal['Año'] = año

        lista_df_demanda.append(df_temporal)

df_demanda_total = pd.concat(lista_df_demanda, ignore_index=True)

print("\n¡Proceso completado!")
print(f"Se procesaron {len(lista_df_demanda)} archivos de demanda.")
display(df_demanda_total.head())

Archivo válido encontrado, procesando: demanda2015.csv
Archivo válido encontrado, procesando: demanda2016.csv
Archivo válido encontrado, procesando: demanda2017.csv
Archivo válido encontrado, procesando: demanda2018.csv
Archivo válido encontrado, procesando: demanda2019.csv
Archivo válido encontrado, procesando: demanda2020.csv
Archivo válido encontrado, procesando: demanda2021.csv
Archivo válido encontrado, procesando: demanda2022.csv
Archivo válido encontrado, procesando: demanda2023.csv
Archivo válido encontrado, procesando: demanda2024.csv

¡Proceso completado!
Se procesaron 10 archivos de demanda.


,Departamento,Municipio,Demanda Real [MWh],Participación [%],Año
0,Antioquia,Abejorral,343.68469,"0,00%",2015
1,Antioquia,Amaga,20732.46459,"0,10%",2015
2,Antioquia,Amalfi,729.33430,"0,00%",2015
3,Antioquia,Andes,1452.13273,"0,01%",2015
4,Antioquia,Angelopolis,1216.19785,"0,01%",2015


In [59]:
demanda_limpia = df_demanda_total[['Año', 'Departamento', 'Demanda Real [MWh]']].copy()

demanda_limpia.rename(columns={'Demanda Real [MWh]': 'Energia_GWh'}, inplace=True)

demanda_limpia['Tipo'] = 'Demanda'

print("DataFrame de Demanda Limpio y Estandarizado")
display(demanda_limpia.head())

DataFrame de Demanda Limpio y Estandarizado


,Año,Departamento,Energia_GWh,Tipo
0,2015,Antioquia,343.68469,Demanda
1,2015,Antioquia,20732.46459,Demanda
2,2015,Antioquia,729.33430,Demanda
3,2015,Antioquia,1452.13273,Demanda
4,2015,Antioquia,1216.19785,Demanda


In [60]:
demanda_limpia['Departamento'].unique()

array(['Antioquia', 'Arauca', 'Atlántico', 'Bogotá. D.C.', 'Bolívar',
       'Boyacá', 'Caldas', 'Caquetá', 'Casanare', 'Cauca', 'Cesar',
       'Chocó', 'Córdoba', 'Cundinamarca', 'Huila', 'La Guajira',
       'Magdalena', 'Meta', 'Nariño', 'Norte De Santander', 'Putumayo',
       'Quindío', 'Risaralda', 'Santander', 'Sucre', 'Tolima',
       'Valle Del Cauca'], dtype=object)

In [61]:
produccion_para_unir = produccion_final.copy()

produccion_para_unir.rename(columns={'Produccion_GWh': 'Energia_GWh'}, inplace=True)

produccion_para_unir['Tipo'] = 'Produccion'

print("--- DataFrame de Producción listo para la unión ---")
display(produccion_para_unir.head())

--- DataFrame de Producción listo para la unión ---


,Año,Departamento,Energia_GWh,Tipo
0,2015,ANTIOQUIA,16495.333354,Produccion
1,2015,ATLÁNTICO,5389.818127,Produccion
2,2015,"BOGOTÁ, D.C.",20.204581,Produccion
3,2015,BOLÍVAR,1205.155267,Produccion
4,2015,BOYACÁ,7394.486413,Produccion


In [62]:
# Unir los dos DataFrames en una única tabla
df_unificado = pd.concat([produccion_para_unir, demanda_limpia], ignore_index=True)

df_unificado.sort_values(by=['Año', 'Departamento'], inplace=True)

# Verificar el resultado final
print("DataFrame Unificado y Listo para Power BI")
display(df_unificado.head())
print("...")
display(df_unificado.tail())

DataFrame Unificado y Listo para Power BI


,Año,Departamento,Energia_GWh,Tipo
0,2015,ANTIOQUIA,16495.333354,Produccion
1,2015,ATLÁNTICO,5389.818127,Produccion
242,2015,Antioquia,343.684690,Demanda
243,2015,Antioquia,20732.464590,Demanda
244,2015,Antioquia,729.334300,Demanda


...


,Año,Departamento,Energia_GWh,Tipo
5745,2024,Valle Del Cauca,335.79444,Demanda
5775,2024,Valle Del Cauca,257.62071,Demanda
5869,2024,Valle Del Cauca,80.24387,Demanda
5878,2024,Valle Del Cauca,56.27750,Demanda
5886,2024,Valle Del Cauca,28.10949,Demanda


In [63]:
#Normalizar la columna de Dpto

df_unificado['Departamento'] = df_unificado['Departamento'].str.title()

print("Departamentos Normalizados")
print(df_unificado['Departamento'].unique())

Departamentos Normalizados
['Antioquia' 'Atlántico' 'Arauca' 'Bogotá, D.C.' 'Bolívar' 'Boyacá'
 'Bogotá. D.C.' 'Caldas' 'Casanare' 'Cauca' 'Cundinamarca' 'Caquetá'
 'Cesar' 'Chocó' 'Córdoba' 'Huila' 'La Guajira' 'Magdalena' 'Meta'
 'Nariño' 'Norte De Santander' 'No Asignado / Menores' 'Putumayo'
 'Quindío' 'Risaralda' 'Santander' 'Sucre' 'Tolima' 'Valle Del Cauca']


In [64]:
df_unificado.head()

,Año,Departamento,Energia_GWh,Tipo
0,2015,Antioquia,16495.333354,Produccion
1,2015,Atlántico,5389.818127,Produccion
242,2015,Antioquia,343.684690,Demanda
243,2015,Antioquia,20732.464590,Demanda
244,2015,Antioquia,729.334300,Demanda


In [65]:
df_unificado.to_csv('datos_energia_unificados.csv', index=False)

In [66]:
#Agrupar por las tres columnas y sumar la energía
df_agrupado = df_unificado.groupby(['Año', 'Departamento', 'Tipo'])['Energia_GWh'].sum().reset_index()

# Opcional: ver el resultado
print(df_agrupado.head(30))

     Año  Departamento        Tipo   Energia_GWh
0   2015     Antioquia     Demanda  2.875311e+06
1   2015     Antioquia  Produccion  1.649533e+04
2   2015        Arauca     Demanda  3.955137e+05
3   2015     Atlántico     Demanda  1.149667e+06
4   2015     Atlántico  Produccion  5.389818e+03
5   2015  Bogotá, D.C.  Produccion  2.020458e+01
6   2015  Bogotá. D.C.     Demanda  2.764870e+06
7   2015       Bolívar     Demanda  1.107525e+06
8   2015       Bolívar  Produccion  1.205155e+03
9   2015        Boyacá     Demanda  1.900287e+06
10  2015        Boyacá  Produccion  7.394486e+03
11  2015        Caldas     Demanda  4.534544e+05
12  2015        Caldas  Produccion  1.568396e+03
13  2015       Caquetá     Demanda  4.503246e+03
14  2015      Casanare     Demanda  5.502151e+04
15  2015      Casanare  Produccion  8.188844e+02
16  2015         Cauca     Demanda  3.913336e+05
17  2015         Cauca  Produccion  1.000682e+03
18  2015         Cesar     Demanda  2.415887e+05
19  2015         Cho

In [67]:
# Guardar el DataFrame unificado directamente Google Drive
df_agrupado.to_csv('/content/drive/MyDrive/datos_energia.csv', index=False)

print("Archivo guardado exitosamente en Google Drive")

Archivo guardado exitosamente en Google Drive
